In [ ]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline


import scipy as sc
import math as ma
from scipy import linalg, optimize, constants, interpolate, special, stats
from math import exp, pow, sqrt, log

import seaborn as sns #spezielle Graphikdarstellungen
import statsmodels.api as sm
import statsmodels.stats.api as sms

#den oberen Teil kennt man schon vom letzten Mal...
#Theano kommt neu dazu - 
#Library für effiziente Berechnungen mit großen Matrizen -> DeepLearning!
import theano 
import tensorflow

 
#Jetzt noch scikit-learn:
#hier stecken viele Funktionalitäten drin, die man gut brauchen kann: 
#Fehlerfunktionen, Standard-Modelltypen, Preprocessing-Algorithmen, Daten... 

import sklearn as sl 
from sklearn import model_selection, metrics, datasets
from sklearn.preprocessing import OneHotEncoder 


Bevor wir richtig loslegen lesen wir nochmal die MNIST-Daten ein

In [ ]:
#Testdaten einlesen

data_file_test = open('./mnist_test.csv', 'r')
data_list_test = data_file_test.readlines() 
data_file_test.close()

#Trainingsdaten einlesen
data_file_train = open('./mnist_train.csv', 'r') 
data_list_train = data_file_train.readlines()
data_file_train.close()

#Check der Dimensionen 
print(len(data_list_test)) #10000 Testdatensätze 
print(data_list_train)
print(len(data_list_train)) #60000 Trainingsdatensätze

In [ ]:
#Alle Trainingsbeispiele durchgehen - erst leere Liste erzeugen
inputs_train = np.empty([60000,784])
targets_train = np.empty([60000,10])
 

k = 0
for record in data_list_train:    
    #an Kommas auftrennen
    all_values_train = record.split(',')
    
    #Skalieren und shiften
    inputs_t = (np.asfarray(all_values_train[1:]) / 255.0 * 0.999) + 0.0001
    
    #Target generieren
    numberoutputs = 10
    targets_t = np.zeros(numberoutputs) + 0.0001
    targets_t[int(all_values_train[0])] = 0.999

    inputs_train[k]  = inputs_t
    targets_train[k] = targets_t
    
    k += 1
    pass

In [ ]:

#Alle Testbeispiele durchgehen - erst leere Liste erzeugen
inputs_test = np.empty([10000,784])
targets_test = np.empty([10000,10])

k = 0
for record in data_list_test:    
    #an Kommas auftrennen
    all_values_test = record.split(',')
    #Skalieren und shiften
    inputs_te = (np.asfarray(all_values_test[1:]) / 255.0 * 0.999) + 0.0001
    #Target generieren
    numberoutputs_test = 10
    targets_te = np.zeros(numberoutputs_test) + 0.0001
    targets_te[int(all_values_test[0])] = 0.999
  
    inputs_test[k]  = inputs_te
    targets_test[k] = targets_te
    
    k += 1
    pass

In [ ]:

#jetzt Daten im Python Format sichern 
#laden ist später jederzeit mit dem Befehl "load" wieder möglich
    
from tempfile import TemporaryFile
mnist_targets_train = TemporaryFile()
mnist_inputs_train = TemporaryFile()
 
mnist_targets_test = TemporaryFile()
mnist_inputs_test = TemporaryFile()
 
np.save(mnist_targets_train, targets_train)
np.save(mnist_inputs_train, inputs_train)
np.save(mnist_targets_test, targets_test)
np.save(mnist_inputs_test, inputs_test)

# 1) Modelle in Keras mit dem Sequential() Modell

Keras erlaubt es mit dem "Sequential" Modell und dem "add" Befehl ein Netz aus verschiedenen Schichten sequentiell aufzubauen

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout


In [ ]:
model = Sequential() # Modellname ist hier "model", kann aber beliebig gewählt werden
model.add(Dense(32, input_dim=784) )
#input_dim gibt die Dimension des Eingangsvektors an und muß spezifiziert werden
model.add(Activation('relu'))

In [ ]:
model = Sequential()
model.add(Dense(32, input_shape=(784,)))

Bevor man ein Modell trainieren kann muß man den Lernprozeß konfigurieren:

In [ ]:
# Für ein multi-class Klassifikationsproblem
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# # Für ein binäres Klassifikationsproblem
# model.compile(optimizer='rmsprop',
#               loss='binary_crossentropy',
#               metrics=['accuracy'])

# # Für ein mean squared error Regressionsproblem
# model.compile(optimizer='rmsprop',
#               loss='mse')

## Graphische Darstellung der Modelltopologie

In [ ]:
from keras.utils import plot_model
plot_model(model, to_file='model.png')

Training des Modelles:
    
Keras Modelle werden auf Numpy arrays mit input Daten und labels trainiert.  
Um ein Modell zu trainieren benutzt man die "fit" Funktion.
Doku: https://keras.io/models/sequential/

In [ ]:
# Single-input Modell mit 2 Klassen (binary classification):


# Dummy Daten

# Training
x_train = np.random.random((1000, 20)) #Input-Vektor mit Dim = 20
y_train = np.random.randint(2, size=(1000, 1)) #Targets: 2 Klassen

# Test
x_test = np.random.random((100, 20))
y_test = np.random.randint(2, size=(100, 1))

# Definiere die Topologie des Netzes
model1 = Sequential()
model1.add(Dense(64, input_dim=20, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(64, activation='relu'))
model1.add(Dropout(0.5))
model1.add(Dense(1, activation='sigmoid'))

# Lernprozeß konfigurieren
model1.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

# Modell trainieren
model1.fit(x_train, y_train,
          epochs=20, #Anzahl Trainingsepochen
          batch_size=128, #Batch-Size
          verbose=0) #print Ausgabe des Trainingsverlaufes 0=nein 1=ja

# Performance auswerten
score = model1.evaluate(x_test, y_test, batch_size=128) #Auswertung des Modelles auf den Testdaten
print(score)
print(model1.metrics_names)

In [ ]:
plot_model(model1, to_file='model1.png')

In [ ]:
# Für ein MLP Modell mit 10 Klassen (categorical classification):

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD #Stochastic Gradient Descent

# Dummy Daten

# Training
x_train = np.random.random((1000, 20)) # Input-Vektor mit Dim = 20
y_train = keras.utils.to_categorical(np.random.randint(10, size=(1000, 1)), num_classes=10) # Target mit 10 Klassen

# Test
x_test = np.random.random((100, 20))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)

# Definiere die Topologie des Netzes
model2 = Sequential()
# Dense(64) ist ein fully-connected Layer mit 64 hidden units.
# in der ersten Schicht muß der input data shape festgelegt werden:
# hier 20-dim. Vektor

model2.add(Dense(64, activation='relu', input_dim=20))
model2.add(Dropout(0.5))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.5))
model2.add(Dense(10, activation='softmax'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #Parameter beim SGD Lernrate, Momentum, ...
model2.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

model2.fit(x_train, y_train,
          epochs=20,
          batch_size=128,
          verbose=0)
score = model2.evaluate(x_test, y_test, batch_size=128)
print(score)
print(model2.metrics_names)

In [ ]:
plot_model(model2, to_file='model.png')

# Attribute eines Modelles

- model.layers: zeigt die verwendeten schichten im Modell an
- model.input und model.output: zeigen welche Inputs/Outputs das Modell generiert
- model.summary(): gibt eine Zusammenfassung

In [ ]:
#print(model1.layers)
print(model1.input)
#print(model1.summary())

In [ ]:
model2.summary()

# 2) Erstes Beispiel in Keras: 3-Layer MLP zur Klassifikation der MNIST-Daten

## MNIST Daten sind schon vorbereitet (Transformation der Daten)

In [ ]:
print(inputs_train.shape)
print(targets_train.shape)

In [ ]:
# Definiere die Topologie des Netzes
modelMNIST = Sequential()
# Dense(64) ist ein fully-connected Layer mit 64 hidden units.
# in der ersten Schicht muß der input data shape festgelegt werden:
# hier 20-dim. Vektor

modelMNIST.add(Dense(20, activation='sigmoid', input_dim=784))
modelMNIST.add(Dense(10, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #Parameter beim SGD Lernrate, Momentum, ...
modelMNIST.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy']) #das ist die "Hit-Rate" = Anteil der richtig klassifizierten Beispiele

historyMNIST=modelMNIST.fit(inputs_train, targets_train,
          epochs=10,
          batch_size=200,
          verbose=0)

In [ ]:
score = modelMNIST.evaluate(inputs_test, targets_test, batch_size=1) #erster Wert loss, zweiter Wert accuracy hier 95%
score

In [ ]:
from keras.utils import plot_model
plot_model(modelMNIST, to_file='modelMNIST.png')

# 3) Visualisierung des Lernens - zeichne Fehler während des Trainings

dazu eine fertige Funktion mit entsprechenden Plot-Routinen

In [ ]:
# updatable plot
# a minimal example (sort of)
from IPython.display import clear_output
class PlotLosses(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        
        self.fig = plt.figure()
        
        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.i += 1
        
        clear_output(wait=True)
        plt.plot(self.x, self.losses, label="loss")
        plt.plot(self.x, self.val_losses, label="val_loss")
        plt.legend()
        plt.show();
        
plot_losses = PlotLosses()

...ein Beispiel

In [ ]:
# Definiere die Topologie des Netzes
modelMNIST = Sequential()
# Dense(64) ist ein fully-connected Layer mit 64 hidden units.
# in der ersten Schicht muß der input data shape festgelegt werden:
# hier 20-dim. Vektor

modelMNIST.add(Dense(20, activation='sigmoid', input_dim=784))
modelMNIST.add(Dense(10, activation='sigmoid'))

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True) #Parameter beim SGD Lernrate, Momentum, ...
modelMNIST.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy']) #das ist die "Hit-Rate" = Anteil der richtig klassifizierten Beispiele

historyMNIST=modelMNIST.fit(inputs_train, targets_train,
          epochs=20,
          batch_size=100,
          verbose=0,
          validation_data=(inputs_test, targets_test),                  
          callbacks=[plot_losses])

## geht auch mit Hilfe der history - allerdings erst, wenn das Training abgeschlossen ist

In [ ]:
# Plotte Training & Validation accuracy (Hit Rate)
plt.plot(historyMNIST.history['accuracy'])
plt.plot(historyMNIST.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plotte Training & Validation loss (hängt von der Loss-Funktion beim Training ab)
plt.plot(historyMNIST.history['loss'])
plt.plot(historyMNIST.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# 4) Ein Beispiel für ein Regressionsproblem

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline


## wir nehmen wieder ein gängiges Benschmark-Beispiel (z.B. von Kaggle)...

## Boston Housing Daten:

The dataset describes 13 numerical properties of houses in Boston suburbs and is concerned with modeling the price of houses in those suburbs in thousands of dollars. As such, this is a regression predictive modeling problem. Input attributes include things like crime rate, proportion of nonretail business acres, chemical concentrations and more.

This is a well-studied problem in machine learning. It is convenient to work with because all of the input and output attributes are numerical and there are 506 instances to work with.



Housing Values in Suburbs of Boston

The medv variable is the target variable.

##  Data description

The Boston data frame has 506 rows and 14 columns.

This data frame contains the following columns:

crim
per capita crime rate by town.

zn
proportion of residential land zoned for lots over 25,000 sq.ft.

indus
proportion of non-retail business acres per town.

chas
Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).

nox
nitrogen oxides concentration (parts per 10 million).

rm
average number of rooms per dwelling.

age
proportion of owner-occupied units built prior to 1940.

dis
weighted mean of distances to five Boston employment centres.

rad
index of accessibility to radial highways.

tax
full-value property-tax rate per $10,000.

ptratio
pupil-teacher ratio by town.

black
1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town.

lstat
lower status of the population (percent).

medv
median value of owner-occupied homes in $1000s.

Source

Harrison, D. and Rubinfeld, D.L. (1978) Hedonic prices and the demand for clean air. J. Environ. Economics and Management 5, 81–102.

Belsley D.A., Kuh, E. and Welsch, R.E. (1980) Regression Diagnostics. Identifying Influential Data and Sources of Collinearity. New York: Wiley.

In [ ]:

# load dataset
dataframe = pd.read_csv("../Daten/housing.csv", delim_whitespace=True, header=None)
dataset = dataframe.values # Keras erwartet Numpy-array keinen Pandas Dataframe!
# split in input (X) und output (Y) Variablen (letzte Spalte enthält das Target!)
X = dataset[:,0:13]
Y = dataset[:,13]

## Split in Training und Validierung

In [ ]:
XTrain_unscaled=X[0:400]
YTrain_unscaled=Y[0:400]
XTest_unscaled=X[400:]
YTest_unscaled=Y[400:]
XTest_unscaled.shape

In [ ]:
#sns.pairplot(dataframe);
#plt.savefig("PairPlotHousing.pdf", dpi=72)

## Daten auf Mittelwert 0 und Varianz 1 skalieren: StandardScaler von ScikitLearn

In [ ]:
from sklearn import preprocessing

scalerX = preprocessing.StandardScaler().fit(XTrain_unscaled)
X_train_scaled= scalerX.transform(XTrain_unscaled)
print(np.mean(X_train_scaled))

scalerXtest = preprocessing.StandardScaler().fit(XTest_unscaled)
X_test_scaled= scalerXtest.transform(XTest_unscaled)
print(np.mean(X_test_scaled))

scalerY = preprocessing.StandardScaler().fit(YTrain_unscaled.reshape(-1,1))
Y_train_scaled= scalerY.transform(YTrain_unscaled.reshape(-1,1))
print(np.mean(Y_train_scaled))

scalerYtest = preprocessing.StandardScaler().fit(YTest_unscaled.reshape(-1,1))
Y_test_scaled= scalerYtest.transform(YTest_unscaled.reshape(-1,1))
print(np.mean(Y_test_scaled))

In [ ]:
## 3-layer MLP für Regression

In [ ]:
modelHousing = Sequential()
modelHousing.add(Dense(50, input_dim=13, kernel_initializer='normal', activation='tanh'))
modelHousing.add(Dense(1, kernel_initializer='normal', activation='linear'))
# Compile model
sgd = SGD(lr=0.1,  momentum=0.9, nesterov=True) #Parameter beim SGD: Lernrate, Momentum, ..
modelHousing.compile(loss='mean_squared_error', optimizer='sgd')


historyHousing=modelHousing.fit(X_train_scaled, Y_train_scaled,
          shuffle=True,
          epochs=50,
          batch_size=100,
          verbose=0,
          validation_data=(X_test_scaled, Y_test_scaled),                  
          callbacks=[plot_losses])

In [ ]:
scoreHousing = modelHousing.evaluate(X_test_scaled, Y_test_scaled, batch_size=1) #erster Wert loss, zweiter Wert accuracy hier 95%
scoreHousing

In [ ]:
y_predicted_scaled=modelHousing.predict(X_test_scaled)

In [ ]:
#Größe und Auflösung der zusammengesetzten Gesamtgraphik
plt.figure(figsize=(6, 6), dpi=90)
plt.plot(y_predicted_scaled,Y_test_scaled,'o')
plt.axis([-2,2,-2,2]); #Achsenausschnitt
plt.xlabel('y-Predicted (scaled)') #Beschriftung x-Achse
plt.ylabel('y-obeserved (scaled)') #Beschriftung y-Achse 
plt.title('Scatter Plot: Model vs. Obs'); #Titel der Graphik


In [ ]:
y_predicted_unscaled=scalerYtest.inverse_transform(y_predicted_scaled)

In [ ]:
#Größe und Auflösung der zusammengesetzten Gesamtgraphik
plt.figure(figsize=(6, 6), dpi=90)
plt.plot(y_predicted_unscaled,YTest_unscaled,'o')
#plt.axis([-2,2,-2,2]); #Achsenausschnitt
plt.xlabel('y-Predicted (scaled)') #Beschriftung x-Achse
plt.ylabel('y-obeserved (scaled)') #Beschriftung y-Achse 
plt.title('Scatter Plot: Model vs. Obs'); #Titel der Graphik



## Aufgabe 1)

- Experimentieren Sie mit den verschiedenen Meta-Parametern im MNIST-Netz (Aktivierungsfunktion, fehlerfunktion, Lernrate, Epochen, Anzahl Hidden-Neuronen,...)
- bauen Sie weitere Schichten in das Netz ein und testen Sie die Performance
- Testen Sie verschiedene Regularisierungsverfahren (Dropout, WeightDecay,...) und fassen Sie das Ergebnis kurz zusammen.


## Aufgabe 2)

- Experimentieren Sie mit den verschiedenen Meta-Parametern im Housing-Netz (Lernrate, Epochen, Anzahl Hidden-Neuronen,...)
- macht es Sinn eine Sigmoid-Aktivierung oder ReLU zu benutzen?
- bauen Sie weitere Schichten in das Netz ein und testen Sie die Performance
- wäre hier EarlyStopping sinnvoll?
- versuchen Sie herauszufinden welche Inputs für das Modell besonders sinnvoll sind (dazu muß man evtl. an den Daten selber etwas tun)

